## Dashboard design
- selector with corresponding scatter plot
- radio buttons with corresponding plot
- table view based on selections
- time series with slider for years

13 sep 2017

In [1]:
# Data
from datetime import date
from random import randint
import pandas as pd

num = [randint(0, 100) for i in range(100)]
df = pd.DataFrame(num, columns=["count"])
df['dates'] = [date(2014+i, 1, 1) for i in range(100)]
df['grp'] = ['A' if x > 80 else 'B' if  x > 30 else 'C' for x in num]
df['item1'] = [randint(0, 100) for i in range(100)]
df['item2'] = [randint(0, 100) for i in range(100)]
df['item3'] = [randint(0, 100) for i in range(100)]

df.head(3)

,count,dates,grp,item1,item2,item3
0,50,2014-01-01,B,78,79,60
1,21,2015-01-01,C,8,45,34
2,55,2016-01-01,B,23,25,65


In [ ]:
#- selector with corresponding scatter plot
# change y-axis in plot
# OK #
from bokeh.layouts import column
from bokeh.plotting import figure, output_file, show ,output_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.models.callbacks import CustomJS
#import numpy as np 

#output_file("bkh_df.html") 
output_notebook()    


#df = pd.DataFrame(np.random.randn(10,3), columns=['num1','y2','y3']) 
df.reset_index(inplace=True) 
df['y'] = df['item1']

source=ColumnDataSource(data=df) 

p1 = figure(plot_width=500, plot_height=300, logo=None, toolbar_location='right') 
p1.circle(source=source, x='count', y='y', size=20, alpha=0.5)  # CHANGED 

callback = CustomJS(args=dict(source=source), code=""" 
        var data = source.get('data'); 
        var f = cb_obj.get('value') 
        data['y'] = data[f]             
        source.trigger('change'); 
    """) 

#y_axis = Select(title="Y:", value='item1', options=['item1', 'item2', 'item3'], callback = callback) 
y_axis = Select(title="Y:", value='item1', options=['item1', 'item2', 'item3']) 
y_axis.js_on_change('value', callback)

layout = column(y_axis, p1)
show(layout) 


In [ ]:
#- table view based on selections
# OK #
import bokeh.embed
import bokeh.io
import bokeh.models
import bokeh.models.widgets
import bokeh.plotting
import pandas as pd
import pandas as pd
import numpy as np
from bokeh.plotting import figure,output_notebook,show,output_file
from bokeh.models import ColumnDataSource, Select, TableColumn, DataTable
from bokeh.layouts import column

output_file("data_table.html")

#df = df.reset_index()

source = ColumnDataSource(df)
original_source = ColumnDataSource(df)
columns = [
    TableColumn(field="count", title="Count"),
    TableColumn(field="dates", title="Dates"),
    TableColumn(field="grp", title="Grp"),
    TableColumn(field="item1", title="item1"),
    TableColumn(field="item2", title="item2"),
    TableColumn(field="item3", title="item3")
]
data_table = DataTable(source=source, columns=columns)

# callback code to be used by all the filter widgets
# requires (source, original_source, country_select_obj, year_select_obj, target_object)
combined_callback_code = """
var data = source.get('data');
var original_data = original_source.get('data');
var grp = grp_select_obj.get('value');
console.log("grp: " + grp);
for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['grp'].length; ++i) {
        if (( grp === 'ALL' || original_data['grp'][i] === grp)) {
            data[key].push(original_data[key][i]);
        }
    }
}
target_obj.trigger('change');
source.trigger('change');
"""

# define the filter widgets, without callbacks for now
#country_list =  df['country'].unique().tolist()
#country_select = Select(title="Country:", value=country_list[0], options=country_list)
grp_select = Select(title="Grp:", value='ALL', options=['ALL','A','B','C'])

# now define the callback objects now that the filter widgets exist
generic_callback = bokeh.models.CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              grp_select_obj=grp_select, 
#              year_select_obj=year_select, 
              target_obj=data_table),
    code=combined_callback_code
)

# finally, connect the callbacks to the filter widgets
#grp_select.callback = generic_callback
grp_select.js_on_change('value', generic_callback)

#year_select.callback = generic_callback

#---
p1 = figure(plot_width=400, plot_height=400)
p1.circle('count', 'item1', source=source, alpha=0.6)
#---

p = column(grp_select, data_table,p1)
show(p)

In [ ]:
#- sine function with slider
# OK #
import numpy as np

from bokeh.layouts import layout
from bokeh.models import CustomJS, Slider, ColumnDataSource, WidgetBox
from bokeh.plotting import figure, output_file, show


#output_file('dashboard.html')
output_notebook()    


tools = 'pan'

def slider():
    x = np.linspace(0, 10, 100)
    y = np.sin(x)

    source = ColumnDataSource(data=dict(x=x, y=y))

    plot = figure(
        y_range=(-10, 10), tools='', toolbar_location=None,
        title="Sliders example")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    callback = CustomJS(args=dict(source=source), code="""
        var data = source.data;
        var A = amp.value;
        var k = freq.value;
        var phi = phase.value;
        var B = offset.value;
        x = data['x']
        y = data['y']
        for (i = 0; i < x.length; i++) {
                y[i] = B + A*Math.sin(k*x[i]+phi);
            }
        
        source.change.emit();
    """)

    amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude", callback=callback, callback_policy='mouseup')
 #   amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
    callback.args["amp"] = amp_slider
 #   amp_slider.js_on_change('amp', callback)

    freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency", callback=callback)
    callback.args["freq"] = freq_slider

    phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase", callback=callback)
    callback.args["phase"] = phase_slider

    offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset", callback=callback)
    callback.args["offset"] = offset_slider

    widgets = WidgetBox(amp_slider, freq_slider, phase_slider, offset_slider)
    return [widgets, plot]

l = layout([slider()])

show(l)


In [3]:
# https://github.com/patricknieto/dashboard/blob/master/test2.py
# OK
# Tabs with graphs and table (no widgets)

import pandas as pd
import numpy as np

from bokeh.io import curdoc,  output_file, show
from bokeh.layouts import row, column, widgetbox, layout
from bokeh.models import ColumnDataSource, DateFormatter
from bokeh.models.widgets import Slider, TextInput, Panel, Tabs, CheckboxGroup, Div
from bokeh.models.widgets import Toggle, DataTable, DateFormatter, TableColumn, Button
from bokeh.plotting import figure,output_notebook,output_file
from bokeh.models.callbacks import CustomJS



output_file('test2.html')
#output_notebook()

files = ['urlvisits-20160823.csv', 'searches-20160823.csv', 'uniqueusers-20160823.csv', 'intab-20160823.csv','urlvisits-20160823.csv']
names = ['URL Visits', 'Searches', 'Unique Users', 'In-Tab Users','Test']


def create_panels(files, names):

    tab_list = []

    for i in range(len(files)):

        df = pd.read_csv('reports/{}'.format(files[i]), sep='\t')
        col1 = df.columns[0]
        col2 = df.columns[1]

        if col1 == 'date':
            df['date'] = pd.to_datetime(df['date'])

            # create a new plot with a datetime axis type
            p = figure(width=800, height=350, x_axis_type="datetime")

            # create 30 day rolling average if plotting dates
            window_size = 30
            window = np.ones(window_size)/float(window_size)
            avg = np.convolve(df[col2], window, 'same')

            source = ColumnDataSource(df)
            #add renderers
            p.line(source.data[col1], avg, color='navy', legend='avg')
            p.circle(source.data[col1], source.data[col2], size=4, color='darkgrey', alpha=0.2, legend='{}'.format(names[i]))

            p.title.text = "One-Month Average of {}".format(names[i])

        else:

            p = figure(width=800, height=350)
            source = ColumnDataSource(df)
            #add renderers
            p.line(source.data[col1], source.data[col2], line_width=2)
            p.circle(source.data[col1], source.data[col2], fill_color="white", size=8)
            p.title.text = "Count of {}".format(names[i])

        # NEW: customize by setting attributes
        p.legend.location = "top_left"
        p.grid.grid_line_alpha=0
        p.xaxis.axis_label = '{}'.format(col1)
        p.yaxis.axis_label = '{}'.format(names[i])
        p.ygrid.band_fill_color="olive"
        p.ygrid.band_fill_alpha = 0.1

        columns = [
            TableColumn(field=c, title=c, formatter=DateFormatter(format='m/d/yy') if np.issubdtype(df[c].dtype, np.datetime64) else None) for c in df.columns
            ]

        data_table = DataTable(source=source, columns=columns, width=400, height=280)

        div = Div(text="""<h2>Quick Info:</h2>
                            <ul>
                            <li>Number of observations: {}</li>
                            <li>Max: {}</li>
                            <li>Min: {}</li>
                            </ul>
                        """.format(df.shape[0], df[col2].max(), df[col2].min()),width=400, height=100)

        info = row(data_table, div) 

        tab_list.append(Panel(child= column(p, info), title=names[i]))

    return tab_list

tabs = Tabs(tabs=create_panels(files, names), width=800)

l= layout([tabs])
show(l)